In [1]:
print("Activating ipynb")
import os 
import numpy as np
import pandas as pd

games_folder_path = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/"
years = [x for x in range(2012,2022)]
for year in years:
    exec(f'games_filename_{year} = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/{year}.csv"')
    # exec(f'games_filename_{year} = {os.path.join(games_folder_path, f"{year}.csv")}')
all_games = os.path.join("C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv")
all_stats = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/stats_sorted.csv"
print(all_games)

Activating ipynb
C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv


In [2]:
for year in years:
    exec(f'results_{year} = pd.read_csv(games_filename_{year}, index_col = False)')
    
all_results_raw = pd.read_csv(all_games)
all_stats_raw = pd.read_csv(all_stats)
all_results = all_results_raw

In [3]:
for year in years:
    exec(f'results_{year}["homeWin"] = results_{year}["homeTeamScore"] > results_{year}["awayTeamScore"]')
    exec(f'y_true_{year} = results_{year}["homeWin"].values')  

all_results['homeWin'] = all_results['homeTeamScore'] > all_results['awayTeamScore']
y_true_all = all_results['homeWin'].values
all_results[:5]

,gameId,year,round,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall,homeWin
0,2012R101,2012,R1,24-Mar-12,Stadium Australia,7:20 PM,38203,Greater Western Sydney,37,Sydney,100,0.0,False
1,2012R102,2012,R1,29-Mar-12,M.C.G.,7:45 PM,78285,Richmond,81,Carlton,125,0.0,False
2,2012R103,2012,R1,30-Mar-12,M.C.G.,7:50 PM,78466,Hawthorn,137,Collingwood,115,0.0,True
3,2012R104,2012,R1,31-Mar-12,M.C.G.,1:45 PM,33473,Melbourne,78,Brisbane Lions,119,0.6,False
4,2012R105,2012,R1,31-Mar-12,Carrara,3:45 PM,12790,Gold Coast,68,Adelaide,137,0.0,False


## Feature Building  
HomeTeam, AwayTeam, Venue, Rainfall, Travel, TeamChangeNumber, TeamChangeGames, LastWeekResult, LastEncounterResult, WinStreak
Form(Last5Games), TeamPlayerTotalRankingScore, BreakBetweenGames, LadderPosition

In [4]:
## Remove uneeded fields
results_clean_2012 = results_2012
results_clean_2012.describe()

,year,attendance,homeTeamScore,awayTeamScore,rainfall
count,207.0,207.000000,207.000000,207.000000,207.000000
mean,2012.0,32747.942029,96.376812,87.347826,2.252174
std,0.0,18335.613605,30.488149,29.316564,5.400394
min,2012.0,6696.000000,37.000000,31.000000,0.000000
25%,2012.0,18922.500000,77.000000,65.500000,0.000000
50%,2012.0,30172.000000,94.000000,86.000000,0.200000
75%,2012.0,41277.000000,114.000000,105.000000,2.000000
max,2012.0,99683.000000,193.000000,180.000000,38.400000


In [5]:
results_clean_2012 = results_2012
results_clean_2012.columns
results_clean_2012 = results_clean_2012.drop(['year', 'date', 'startTime', 'attendance'], axis=1)
results_clean_2012.columns

a_2012 = results_clean_2012

## Build Y value 
Result of game based on Home Team (Win = 2, Draw = 1, Lose = 0)

In [6]:
def homeWin(row):
    homeScore = row["homeTeamScore"]
    awayScore = row["awayTeamScore"]
    row['homeWin'] = 2 if homeScore > awayScore else 0 if homeScore < awayScore else 1 
        
    return row
  
a_2012 = a_2012.apply(homeWin, axis=1)
y_true_2012 = a_2012['homeWin']


## Add Previous Game Result
Set default dict to store results for each team
Only a boolean value, draws only account for ~1% of all games 
Default to false
If the last game was a win set to true
# TODO 
add draws in and see if there is a difference

In [7]:
a_2012["homeTeamLastWin"] = False
a_2012["awayTeamLastWin"] = False

from collections import defaultdict
won_last = defaultdict(bool)

def teamLastWin(row):
    home_team = row["homeTeam"]
    away_team = row["awayTeam"]
    row["homeTeamLastWin"] = won_last[home_team]
    row["awayTeamLastWin"] = won_last[away_team]
    
    won_last[home_team] = row["homeWin"] == "Win"
    won_last[away_team] = row["homeWin"] == "Loss"
    
    return row
  
a_2012 = a_2012.apply(teamLastWin, axis=1)

## One Hot Encoding
The tree models used for feature engineering cannot take in string inputs
Value encoding can be used to change teams to int values, but then the model will read this as distance i.e. the team with value 1 will be read as being closer to the team with value 2 than the team with value 18. 
One Hot Encoding creates a bool variable for each team which is 1 when the team is playing.

# TODO
See if the model performs better with the simple label encoding. 
One hot encoding creates an issue with decision trees as it creates lots of "empty" splits, there will now be 34 variables which the model will split on 0, which can skew the model.

In [8]:
# One Hot Encoding Teams
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(all_results["homeTeam"].values)

home_teams_2012 = encoding.transform(a_2012["homeTeam"].values)
away_teams_2012 = encoding.transform(a_2012["awayTeam"].values)

all_teams_2012 = np.vstack([home_teams_2012, away_teams_2012]).T
all_teams_2012[:5], all_teams_2012.shape

oneHot = OneHotEncoder()
X_teams_2012 = oneHot.fit_transform(all_teams_2012).todense()
X_teams_2012.shape

(207, 36)

## One Hot Encoding Venues
Same issues as above, but simplified slightly as there are only 23 stadiums

In [9]:
# One Hot Encoding Venues
encoding = LabelEncoder()
encoding.fit(all_results["venue"].values)
venues = encoding.transform(a_2012['venue'].values)

all_venues = np.vstack(venues).T
venues = venues.reshape(-1,1)
venues[:5], venues.shape

oneHot = OneHotEncoder()
X_venues = oneHot.fit_transform(venues).todense()
X_venues = np.asarray(X_venues)
X_venues.shape

(207, 16)

Computed Variables
These variables did not appear in the original data set but were added later
Travel Data: Used to determine the impact of travel on teams
             Determined by comparing the venue of the upcoming match to the teams home location
Break: Finds the break between a teams last and next game
Team Changes: The number of players that are different in the from the previous match
              The number of games lost/gained from these changes
              The expected score lost/gained from the incoming players
              # TODO split games out and games in into separate variables
Team Composition: The average/total score of all the selected players
                  Done this way to reduce number of features
Previous 5 games result: Totals each teams previous games results

             

In [13]:
## Travel Distance
import AFL_Stats.travel_distance as td
def travel(row):
    row['awayTravel'] = td.team_travel(row['awayTeam'], row['venue'])
    row['homeTravel'] = td.team_travel(row['homeTeam'], row['venue'])
    return row
a_2012 = a_2012.apply(travel, axis=1)

a_2012[:10]

,gameId,round,venue,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall,homeWin,homeTeamLastWin,awayTeamLastWin,awayTravel,homeTravel
0,2012R101,R1,Stadium Australia,Greater Western Sydney,37,Sydney,100,0.0,0,False,False,0,0
1,2012R102,R1,M.C.G.,Richmond,81,Carlton,125,0.0,0,False,False,0,0
2,2012R103,R1,M.C.G.,Hawthorn,137,Collingwood,115,0.0,2,False,False,0,0
3,2012R104,R1,M.C.G.,Melbourne,78,Brisbane Lions,119,0.6,0,False,False,0,0
4,2012R105,R1,Carrara,Gold Coast,68,Adelaide,137,0.0,0,False,False,3,0
5,2012R106,R1,Subiaco,Fremantle,105,Geelong,101,0.0,2,False,False,3,0
6,2012R107,R1,Docklands,North Melbourne,102,Essendon,104,0.6,0,False,False,0,0
7,2012R108,R1,Docklands,Western Bulldogs,87,West Coast,136,0.0,0,False,False,4,0
8,2012R109,R1,Football Park,Port Adelaide,89,St Kilda,85,0.0,2,False,False,2,0
9,2012R201,R2,Gabba,Brisbane Lions,63,Carlton,154,0.0,0,False,False,3,0


In [19]:
# Team Similarity
from AFL_Stats.team_changes import team_similarity

a_2012['homeTeamChanges'] = 0
a_2012['awayTeamChanges'] = 0
a_2012['homeGameDifference'] = 0
a_2012['awayGameDifference'] = 0

def getPreviousGame(round, team):
    gameId = a_2012.query('round == @round and (homeTeam == @team or awayTeam == @team)')
    return gameId.gameId.values[0]

def getChanges(row):
    if row['round'] == 'R1':
        return row
    homeTeam = row['homeTeam']
    awayTeam = row['awayTeam']
    round = row['round']
    gameId = row['gameId']

    prevGameHomeTeam = getPreviousGame(round, homeTeam)
    prevGameAwayTeam = getPreviousGame(round, awayTeam)
    homeChanges = team_similarity(gameId, prevGameHomeTeam, homeTeam)
    awayChanges = team_similarity(gameId, prevGameAwayTeam, awayTeam)

getPreviousGame('R2', 'Carlton')

2012R201


In [14]:
# Replace Teams and venues with OHE values
a_2012_ohe = a_2012.drop(['homeTeam','awayTeam','venue','homeWin', 'homeTeamScore','awayTeamScore', 'round'], axis=1)
a_2012_ohe.columns
a_2012_ohe = np.hstack([a_2012_ohe, X_teams_2012, X_venues])

a_2012_ohe

matrix([['2012R101', 0.0, False, ..., 0.0, 0.0, 0.0],
        ['2012R102', 0.0, False, ..., 0.0, 0.0, 0.0],
        ['2012R103', 0.0, False, ..., 0.0, 0.0, 0.0],
        ...,
        ['2012PF01', 0.0, False, ..., 0.0, 0.0, 0.0],
        ['2012PF02', 1.0, False, ..., 0.0, 0.0, 0.0],
        ['2012GF01', 8.4, False, ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(random_state=26)
y_true_2012 = np.asarray(y_true_2012)
a_2012_ohe_in = np.asarray(a_2012_ohe)
scores = cross_val_score(clf, a_2012_ohe_in, y_true_2012)

print("F1: {0:0.4f}".format(np.mean(scores)))

F1: 0.6333


C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
